In [1]:
import os

import sys
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\src")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN")
sys.path.append("C:\\Users\\User\\PycharmProjects\\GIEBN\\venv")

path = os.getcwd()

curr_path = path.replace('\\', '/')
head_place = curr_path.find('GIEBN')
main_path = curr_path[:head_place]+'GIEBN'
main_path

'c:/Users/User/PycharmProjects/GIEBN'

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeClassifier

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE

from auxiliary.auxiliary import construct_by_kmeans

from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
import yodo.yodo
import matplotlib.pyplot as plt
from pgmpy.estimators import PC, BicScore, K2Score
from pgmpy.inference import VariableElimination
from bamt.networks.discrete_bn import DiscreteBN
from bamt_inherited.networks.discrete import DiscreteBNGI
import shap
from sklearn import metrics as m
from tqdm import tqdm
from itertools import product
import optuna
from copy import copy

In [3]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [4]:
%run ../common_code.ipynb

In [5]:
data = pd.read_csv("../../datasets/abalone.data", sep=',', names=['sex', 'length', 'diameter', 'height', 'W_weight', 'S_weight', 'viscera_weight', 'shell_weight', 'rings'])
data = data[data["sex"] != "I"]
data["sex"] = data["sex"].replace({"F": 1, "M": 0})
data = data.reset_index().drop(columns=["index"])
n_features = data.shape[1]-1
n = data.shape[0]
target = "sex"
features = data.columns.difference([target]).tolist()

data.head(10)

,sex,length,diameter,height,W_weight,S_weight,viscera_weight,shell_weight,rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,1,0.530,0.415,0.150,0.7775,0.2370,0.1415,0.330,20
5,1,0.545,0.425,0.125,0.7680,0.2940,0.1495,0.260,16
6,0,0.475,0.370,0.125,0.5095,0.2165,0.1125,0.165,9
7,1,0.550,0.440,0.150,0.8945,0.3145,0.1510,0.320,19
8,1,0.525,0.380,0.140,0.6065,0.1940,0.1475,0.210,14
9,0,0.430,0.350,0.110,0.4060,0.1675,0.0810,0.135,10


In [102]:
data_corr = add_extra_features(data, features, extra_size=0.75, mode='prod')

In [103]:
data_dict_corr = {}

data_tv, data_dict_corr['test'] = train_test_split(data_corr, test_size=0.15, random_state=42,
                                      stratify=data_corr[target])

data_dict_corr['train'], data_dict_corr['val'] = train_test_split(data_tv, test_size=0.15, random_state=42,
                                      stratify=data_tv[target])

features_corr = list(data_corr.columns.difference([target]))

In [104]:
def objective(trial, features, target, data_dict):
    n_estimators= trial.suggest_int("n_estimators", low=params_net['n_estimators'][0], high=params_net['n_estimators'][1], step=1)
    max_depth= trial.suggest_int("max_depth", low=params_net['max_depth'][0], high=params_net['max_depth'][1], step=1)
    learning_rate=trial.suggest_float("learning_rate", low=params_net['learning_rate'][0], high=params_net['learning_rate'][1],log=True)
    
    clf = GradientBoostingClassifier(random_state=42, n_estimators=n_estimators,
                                     max_depth=max_depth, learning_rate=learning_rate)
    clf.fit(data_dict['train'][features], data_dict['train'][target])
    return score_classif(clf, data_dict['val'][features], data_dict['val'][target])

params_net = {
    'n_estimators': [10, 100],
    'max_depth': [3, 10],
    'learning_rate':[1e-5, 1]
}


# Initial model for error calculation

In [105]:
best_base_params = optunize_model(objective, features_corr, target, data_dict_corr)

  0%|          | 0/30 [00:00<?, ?it/s]

In [106]:
best_base_params

{'n_estimators': 42, 'max_depth': 3, 'learning_rate': 0.018046460876365543}

In [107]:
model_corr = GradientBoostingClassifier(random_state=42, **best_base_params)
#model = LogisticRegression()
model_corr.fit(data_dict_corr['train'][features_corr], data_dict_corr['train'][target])

data_errors_corr = measure_error(data_dict_corr['val'], model_corr, features_corr, target, error_col_name='inconf_error', task='classif')

In [108]:
n_features_corr = len(features_corr)
disc_data_corr, pipeline_data_corr = discretize(data_errors_corr, 
                                      features_corr+["inconf_error"], icat=None, icont=list(range(n_features_corr+1)))

categories_corr = extract_categories(disc_data_corr)
encoder = OneHotEncoder(sparse=False)
disc_data_encoded_corr = pd.DataFrame(encoder.fit_transform(X=disc_data_corr), columns=categories_corr, dtype='uint8')
disc_data_encoded_corr.head(1)

,length_0,length_1,length_2,diameter_0,diameter_1,diameter_2,height_0,height_1,height_2,W_weight_0,...,extra_22_2,extra_23_0,extra_23_1,extra_23_2,extra_24_0,extra_24_1,extra_24_2,inconf_error_0,inconf_error_1,inconf_error_2
0,0,0,1,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1


In [109]:
feat_to_grad_corr = {}
for feat in features_corr+['inconf_error']:
    feat_to_grad_corr[feat]=[grad for grad in disc_data_encoded_corr.columns if feat in grad]


metric_dict = {
    'ROC AUC': lambda x, y: m.roc_auc_score(x,y)*100,
    'log_loss': lambda x, y: m.log_loss(x, y)
}

gbn_fs = DiscreteBNGI('')

gbn_training_pipeline(gbn_fs, disc_data_encoded_corr, 'inconf_error', categories_corr)

2025-05-04 21:31:09,777 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,778 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,779 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,781 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,783 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,785 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,787 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,788 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,789 | ERROR    | GraphUtils.py-nodes_types-0034 | Unsupported data type. Dtype: uint8
2025-05-04 21:31:09,790 | ERROR    | GraphUtil

  0%|          | 0/1000000 [00:00<?, ?it/s]

# Univariate Feature selector

In [110]:
univ_sel = SelectKBest(k=len(features_corr))
univ_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_univ = pd.DataFrame({'Feature': univ_sel.feature_names_in_, 'Metric value': univ_sel.pvalues_})

# RFE on decision tree

In [111]:
rfe_sel = RFE(estimator=DecisionTreeClassifier(random_state=42, max_depth=10,), n_features_to_select=1)
#rfe_sel = RFE(estimator=RandomForestClassifier(random_state=42, max_depth=10, n_estimators=50), n_features_to_select=1, )
rfe_sel.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
df_val_corr_rfe = pd.DataFrame({'Feature': rfe_sel.feature_names_in_, 'Metric value': rfe_sel.ranking_})

# Lasso from Logistic Regression with L1 penalty

In [112]:
lsvc = LogisticRegression(C=0.5, penalty="l1", solver='liblinear')
lsvc.fit(data_dict_corr['val'][features_corr], data_dict_corr['val'][target])
lasso_selector = SelectFromModel(lsvc, prefit=True)

In [113]:
lasso_support = [x for i, x in enumerate(features_corr) if lasso_selector.get_support()[i]]
lasso_support

['S_weight', 'extra_11', 'rings']

## GBN selector

In [114]:
df_val_corr_bn=error_based_importance_features_bn(gbn_fs, 'inconf_error', features_corr, feat_to_grad_corr).fillna(0)

100%|██████████| 32/32 [00:12<00:00,  2.55it/s]


## GBN selector run

In [115]:
gbn_res = elimination_experiment(model_corr, data_dict_corr, metric_dict, df_val_corr_bn, direction='asc')

100%|██████████| 32/32 [00:46<00:00,  1.45s/it]


In [116]:
gbn_res.head()

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,32,-,57.805,0.676,0,0
1,31,extra_21,57.752,0.676,-0.092,0.053
2,30,extra_3,57.767,0.676,-0.065,0.002
3,29,extra_11,57.482,0.676,-0.558,0.029
4,28,extra_24,57.775,0.676,-0.052,0.007


In [117]:
get_best_record(gbn_res, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
5,27,shell_weight,58.966,0.676,2.009,0.017


In [118]:
get_best_record(gbn_res, 'log_loss', mode='min')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,32,-,57.805,0.676,0,0
1,31,extra_21,57.752,0.676,-0.092,0.053
2,30,extra_3,57.767,0.676,-0.065,0.002
3,29,extra_11,57.482,0.676,-0.558,0.029
4,28,extra_24,57.775,0.676,-0.052,0.007
5,27,shell_weight,58.966,0.676,2.009,0.017
6,26,extra_20,58.902,0.676,1.898,0.045
7,25,extra_18,58.718,0.676,1.579,0.049
8,24,extra_15,58.75,0.676,1.635,0.05
9,23,extra_12,58.693,0.676,1.537,0.054


In [119]:
df_corr_excluded_asc_univ=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_univ, 'asc')
df_corr_excluded_asc_univ.head()

100%|██████████| 32/32 [00:41<00:00,  1.29s/it]


,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,32,-,57.805,0.676,0,0
1,31,diameter,57.838,0.676,0.058,0.052
2,30,extra_16,58.596,0.676,1.368,0.024
3,29,extra_20,58.632,0.676,1.431,0.028
4,28,extra_5,58.016,0.676,0.365,0.025


In [120]:
get_best_record(df_corr_excluded_asc_univ, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
18,14,extra_23,60.778,0.676,5.142,-0.072


In [121]:
get_best_record(df_corr_excluded_asc_univ, 'log_loss', mode='min')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
19,13,extra_19,60.247,0.675,4.225,-0.095
20,12,extra_4,60.448,0.675,4.572,-0.098
21,11,W_weight,60.457,0.675,4.588,-0.109
22,10,extra_11,60.244,0.675,4.219,-0.138


In [122]:
df_corr_excluded_rfe=elimination_experiment(model_corr, data_dict_corr, metric_dict,
                       df_val_corr_rfe, 'desc')
df_corr_excluded_rfe.head()

100%|██████████| 32/32 [00:44<00:00,  1.38s/it]


,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,32,-,57.557,0.676,0,0
1,31,viscera_weight,58.1,0.676,0.943,-0.0
2,30,extra_5,58.076,0.676,0.901,0.008
3,29,shell_weight,57.582,0.676,0.044,0.066
4,28,extra_2,57.653,0.676,0.167,0.015


In [123]:
get_best_record(df_corr_excluded_rfe, 'ROC AUC')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
20,12,extra_17,60.215,0.676,4.618,0.056


In [124]:
get_best_record(df_corr_excluded_rfe, 'log_loss', mode='min')

,n_features,last_excluded,ROC AUC,log_loss,"ROC AUC_change, %","log_loss_change, %"
0,32,-,57.557,0.676,0,0
1,31,viscera_weight,58.1,0.676,0.943,-0.0
2,30,extra_5,58.076,0.676,0.901,0.008
3,29,shell_weight,57.582,0.676,0.044,0.066
4,28,extra_2,57.653,0.676,0.167,0.015
5,27,extra_11,58.061,0.676,0.876,0.034
13,19,extra_16,57.876,0.676,0.554,0.029
14,18,extra_13,57.876,0.676,0.554,0.029
15,17,extra_20,58.656,0.676,1.909,0.007
16,16,extra_22,58.549,0.676,1.724,0.062


In [125]:
single_run(model_corr, data_dict_corr, lasso_support, target, metric_dict)

[57.057, 0.677]

In [ ]:
#logging
0.5 random, base 79.953, gbn 80.075, univ 79.953, rfe 79.953, lasso: 79.944
0.75 random, base 80.119, gbn 80.55	, univ 80.119, rfe 80.119, lasso: 80.242

0.5 noisy base 80.42, gbn 80.58	, univ 80.42, rfe 80.578, lasso: 80.515
0.75 noisy base 80.264, gbn 80.311, univ 80.264, rfe 80.264, lasso: 80.633

0.5 prod base 80.007, gbn 80.203	, univ 80.022, rfe 80.448, lasso: 80.422
0.75 prod base 80.36, gbn 80.422	, univ 80.549, rfe 80.542, lasso: 80.316
